In [1]:
%run "/home/jovyan/work/workspace/functions/create_spark_session.ipynb"

Spark Version: 3.5.0


In [2]:
%run "/home/jovyan/work/workspace/functions/paths.ipynb"

In [3]:
%run "/home/jovyan/work/workspace/functions/dates.ipynb"

In [4]:
from pyspark.sql.functions import split,col,explode, upper, trim , when

In [16]:
df = spark.read.format('delta').load(f'{silver}channel_group')

In [17]:
df.createOrReplaceTempView('channel')
spark.sql(''' describe channel ''').toPandas()

,col_name,data_type,comment
0,tpchannel,string,None
1,tpgroup,string,None
2,tptrade,string,None
3,dtload,date,None


In [18]:
df = (

    df.withColumn('split', split(col('tpchannel'),'/'))
    .withColumn('tpchannel', explode(col('split')))
    .withColumn('tpchannel', trim(col('tpchannel')))    
    .drop('split')
    .withColumn("cdgroup", 
      when(col("tpgroup") == "GOV & MILITARY", 0)
     .when(col("tpgroup") == "SERVICES", 1)
     .when(col("tpgroup") == "ENTERTAINMENT", 2)
     .when(col("tpgroup") == "OTHER", 3)
     .when(col("tpgroup") == "GROCERY", 4)
     .when(col("tpgroup") == "ACADEMIC", 5))
    .select(['tpchannel','cdgroup', 'tpgroup', 'tptrade', 'dtload'])
)

In [13]:
df.write.partitionBy('dtload').format('delta').mode('overwrite').save(f'{gold}channel_group')